In [69]:
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data.dataset import Dataset 
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms,models
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
import shutil
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from sklearn.metrics import f1_score   
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

In [70]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

In [71]:
state_dict=torch.load('regnet.pth')

In [88]:
test_dir =os.path.join("Data",os.path.join("test","flooded"))

In [89]:
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
     transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [90]:
class TestDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_files = os.listdir(folder_path)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_file = self.image_files[index]
     
        image_path =os.path.join(self.folder_path, image_file)
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [91]:
test_dataset = TestDataset(test_dir, transforms_test)

In [92]:
test_dataloader = DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers=8)

In [93]:
os.path.join(test_dataset.folder_path, os.listdir(test_dataset.folder_path)[0])

'Data\\test\\flooded\\100.jpg'

In [94]:
model2 =models.regnet_x_1_6gf()
num_features = model2.fc.in_features   
model2.fc = nn.Linear(num_features, 2) 
model2.load_state_dict(torch.load("regnet2.pt"))

<All keys matched successfully>

In [95]:
model2.eval()
results=[]
with torch.no_grad():
        for inputs in test_dataloader:
                inputs = inputs.to(device) 
                outputs = model2(inputs)
                _, preds = torch.max(outputs, 1)
                results.append(preds)
                #0 flooded , 1 non flooded 
                results = torch.cat(results, dim=0)
                results = results.to('cpu').numpy().flatten()
    

RuntimeError: DataLoader worker (pid(s) 9320, 24984, 9464, 17644, 3028, 21936, 20720, 8396) exited unexpectedly

In [ ]:
with open('preds.txt', 'w') as f:
    # Loop over the values
    for value in results:
        # Check if the value is equal to 0
        if value == 0:
            # Write "flooded" to the text file
            f.write('flooded\n')
        else:
            # Write "non flooded" to the text file
            f.write('non flooded\n')


In [ ]:
with open('output.txt', 'w') as f:
    # Loop over the values
    for value in results:
        # Check if the value is equal to 0
        if value == 0:
            # Write "flooded" to the text file
            f.write('0\n')
        else:
            # Write "non flooded" to the text file
            f.write('1\n')
